---
title: "Notebook 1 — Data Acquisition & Organization"
format: html
---

### DS 2023 — Final Project  
### Author: Matthew Benson  
### Topic: Night Sky Darkness Degradation in Southern Utah / Powell Point


## 1. Overview

This notebook covers all aspects of how the data was:

- acquired  
- prepared  
- validated  
- organized into tidy dataframes  

My Two Data Sources:

### **(A) Globe at Night Dataset [Referred to as GaN] (2006–2024)**
A global citizen-reporting dataset where volunteers record *Limiting Magnitude* — a measure of sky darkness.

### **(B) NPS Night Sky Monitoring Data (Powell Point)**
Professional measurements collected using scientific instruments.

---

## 2. Data Sources

### 2.1 Globe at Night (GaN)
- **Producer:** NOAO / NSF’s NOIRLab  
- **Format:** CSV 
- **Years Used:** 2006–2024 - 19 total combined datasets  
- **Access Method:** Downloaded from the GaN public repository  
- **Primary Measurement:** `LimitingMag` (higher = darker sky)  

### 2.2 National Park Service Night Sky Program (NPS)
- **Producer:** NPS Natural Sounds & Night Skies Division  
- **Format:**  CSV 
- **Years:** 2007, 2015, 2025 | Specific to Powell Point
- **Metrics:** Zenith brightness, percent stars visible, synthetic SQM  

---

## 3. Code USed to Import and Clean/Create Dataframes 

In [ ]:
import pandas as pd
import glob

# GaN yearly files
filepaths = glob.glob("GaN*.csv")  

dfs = []

for fp in filepaths:
    # Extract year 
    year = int("".join(filter(str.isdigit, fp)))

    df = pd.read_csv(fp)

    # Clean the LimitingMag column
    df["LimitingMag"] = pd.to_numeric(df.get("LimitingMag"), errors="coerce")

    # Adding the year column
    df["Year"] = year

    # Preserving magnitude values
    df = df[df["LimitingMag"].between(0, 9)]

    dfs.append(df)

# Combine all years into tidy dataframe
gan_all = pd.concat(dfs, ignore_index=True)

# Final cleaning
gan_all["LimitingMag"] = pd.to_numeric(gan_all["LimitingMag"], errors="coerce")
gan_all = gan_all[gan_all["LimitingMag"].between(0, 9)]

#-------------------------------
# FIlter to regional data- Utah
#-------------------------------
lon_min, lon_max = -115.5, -108.5
lat_min, lat_max =  34.5,   40.0

gan_region = gan_all[
    (gan_all["Longitude"].between(lon_min, lon_max)) &
    (gan_all["Latitude"].between(lat_min, lat_max))
].copy()

gan_region.head()

## Data Shape and Identifying Data Was Cleaned Correctly

In [ ]:
# Identifying sahpe as well as value counts to better grasp data

print("Global dataset shape:", gan_all.shape)
print("Regional dataset shape:", gan_region.shape)

gan_all["Year"].value_counts().sort_index()

## Powell Point Dataset 

In [ ]:
# Scientific data copied straight from the site's geogrpahical point
data = {
    "Site": ["Powell Memorial", "Powell Memorial", "Powell Point"],
    "Date": ["2007-09-12", "2015-10-12", "2025-02-06"],
    "Year": [2007, 2015, 2025],
    "AllSky_LPR": [0.10, 0.10, 0.17],
    "Zenith_mag": [21.34, 21.68, 22.05],
    "Zenith_LimMag": [7.3, 7.0, None],
    "Synthetic_SQM": [21.8, 21.42, 21.74],
    "Percent_Stars_Visible": [96.6, 97.3, 91.4]
}

powell_nps = pd.DataFrame(data)
powell_nps

## 4. COLS Table for GaN data

In [ ]:
import pandas as pd

cols_table = pd.DataFrame({
    "Column": [
        "Latitude",
        "Longitude",
        "LimitingMag",
        "Year",
        "LocalDate",
        "LocalTime",
        "CloudCover",
        "SkyComment",
        "ObsType",
        "Elevation(m)"
    ],
    "Dataset": [
        "GaN",
        "GaN",
        "GaN",
        "Derived",
        "GaN",
        "GaN",
        "GaN",
        "GaN",
        "GaN",
        "GaN"
    ],
    "Meaning": [
        "Observer latitude (degrees)",
        "Observer longitude (degrees)",
        "Naked-eye limiting magnitude (higher = darker sky)",
        "Year extracted from filename; used for trend analysis",
        "Local date of observation",
        "Local time of observation",
        "Estimated cloud coverage at time of observation",
        "Observer comments about sky conditions",
        "Type of observation (e.g., Globe at Night, SQM)",
        "Elevation of observation point (meters)"
    ]
})

cols_table

## COLS Table for Powell Point data

In [ ]:
import pandas as pd

powell_cols = pd.DataFrame({
    "Column": [
        "Site",
        "Date",
        "Year",
        "AllSky_LPR",
        "Zenith_mag",
        "Zenith_LimMag",
        "Synthetic_SQM",
        "Percent_Stars_Visible"
    ],
    "Dataset": [
        "NPS",
        "NPS",
        "NPS",
        "NPS",
        "NPS",
        "NPS",
        "NPS",
        "NPS"
    ],
    "Meaning": [
        "Name of observation site",
        "Date when measurement was taken",
        "Year extracted from date",
        "All-sky light pollution ratio",
        "Calibrated zenith sky brightness (mag/arcsec²)",
        "Limiting magnitude at zenith (faintest visible star)",
        "Sky Quality reading",
        "Estimated percent of stars visible"
    ]
})

powell_cols